In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] =  '0' #'3,2,1,0'
import sys
sys.path.append("..")
from common import *
from data   import *



##----------------------------------------
from model_seresnext50 import *

TkAgg
@common.py:  
	set random seed
		SEED=35202
	set cuda environment
		torch.__version__              = 0.4.1
		torch.version.cuda             = 9.0
		torch.backends.cudnn.version() = 7005
		os['CUDA_VISIBLE_DEVICES']     = 0
		torch.cuda.device_count()      = 1



In [2]:

def test_augment(drawing,label,index, augment):
#     cache = Struct(data = drawing.copy(), label = label, index=index)

    #<todo> ... different test-time augment ...

    image = drawing_to_image_with_color_v2(drawing, 96, 96)
    return image, label, None




##############################################################################################

#generate prediction npy_file
def make_npy_file_from_model(checkpoint, mode, split, augment, out_test_dir, npy_file):

    ## setup  -----------------
    # os.makedirs(out_test_dir +'/backup', exist_ok=True)
    # backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.test.%s.zip'%IDENTIFIER)

    log = Logger()
    log.open(out_test_dir +'/log.submit.txt',mode='a')
    log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
    log.write('\tSEED         = %u\n' % SEED)
    log.write('\tPROJECT_PATH = %s\n' % PROJECT_PATH)
    log.write('\tout_test_dir = %s\n' % out_test_dir)
    log.write('\n')


    ## dataset ----------------------------------------
    log.write('** dataset setting **\n')
    batch_size  = 256 #256 #512

    test_dataset = DoodleDataset(mode, split,
                              lambda drawing, label, index : test_augment(drawing, label, index, augment),)
    test_loader  = DataLoader(
                        test_dataset,
                        sampler     = SequentialSampler(test_dataset),
                        batch_size  = batch_size,
                        drop_last   = False,
                        pin_memory  = True,
                        collate_fn  = null_collate)

    assert(len(test_dataset)>=batch_size)
    log.write('test_dataset : \n%s\n'%(test_dataset))
    log.write('\n')


    ## net ----------------------------------------
    log.write('** net setting **\n')
    net = Net().cuda()

    log.write('%s\n\n'%(type(net)))
    log.write('\n')



    if 1:
        log.write('\tcheckpoint = %s\n' % checkpoint)
        net.load_state_dict(torch.load(checkpoint, map_location=lambda storage, loc: storage))


        ####### start here ##########################
        criterion = softmax_cross_entropy_criterion
        test_num  = 0
        probs    = []
        truths   = []
        losses   = []
        corrects = []

        net.set_mode('test')
        for input, truth, cache in test_loader:
            print('\r\t',test_num, end='', flush=True)
            test_num += len(truth)

            with torch.no_grad():
                input = input.cuda()
                logit = data_parallel(net,input)
                prob  = F.softmax(logit,1)
                probs.append(prob.data.cpu().numpy())


                if mode=='train': # debug only
                    truth = truth.cuda()
                    loss    = criterion(logit, truth, False)
                    correct = metric(logit, truth, False)

                    losses.append(loss.data.cpu().numpy())
                    corrects.append(correct.data.cpu().numpy())
                    truths.append(truth.data.cpu().numpy())


        assert(test_num == len(test_loader.sampler))
        print('\r\t',test_num, end='\n', flush=True)
        prob = np.concatenate(probs)

        if mode=='train': # debug only
            correct = np.concatenate(corrects)
            truth   = np.concatenate(truths).astype(np.int32).reshape(-1,1)
            loss    = np.concatenate(losses)
            loss    = loss.mean()
            correct = correct.mean(0)
            top = [correct[0], correct[0]+correct[1], correct[0]+correct[1]+correct[2]]
            precision = correct[0]/1 + correct[1]/2 + correct[2]/3
            print('top      ', top)
            print('precision', precision)
            print('')
    #-------------------------------------------


    np.save(npy_file, np_float32_to_uint8(prob))
    print(prob.shape)
    log.write('\n')






def prob_to_csv(prob, key_id, csv_file):
    top = np.argsort(-prob,1)[:,:3]
    word = []
    for (t0,t1,t2) in top:
        word.append(
            CLASS_NAME[t0] + ' ' + \
            CLASS_NAME[t1] + ' ' + \
            CLASS_NAME[t2]
        )
    df = pd.DataFrame({ 'key_id' : key_id , 'word' : word}).astype(str)
    df.to_csv(csv_file, index=False, columns=['key_id', 'word'], compression='gzip')



def npy_file_to_sbmit_csv(mode, split, npy_file, csv_file):
    print('NUM_CLASS', NUM_CLASS)
    complexity='simplified'

    if mode=='train':
        raise NotImplementedError

    if mode=='test':
        assert(NUM_CLASS==340)
        global TEST_DF

        if TEST_DF == []:
            TEST_DF = pd.read_csv(DATA_DIR + '/csv/test_%s.csv'%(complexity))
        key_id = TEST_DF['key_id'].values


    prob = np_uint8_to_float32(np.load(npy_file))
    print(prob.shape)

    prob_to_csv(prob, key_id, csv_file)
















#################################################################################################3

def run_test_fold():

    mode  = 'test' #'train'
    configures =[
         Struct(
            split        = '<NIL>', #'valid_0', #
            out_test_dir = '../split/test',
            checkpoint   = '../../output/backup/00126000_model_96color_r2.pth',
         ),
    ]


    for configure in configures:
        split        = configure.split
        out_test_dir = configure.out_test_dir
        checkpoint   = configure.checkpoint
        augment      = 'null'

        npy_file = out_test_dir + '/%s-%s.prob.uint8.npy'%(mode,augment)
        csv_file = out_test_dir + '/%s-%s.submit_885.csv.gz'%(mode,augment)

        make_npy_file_from_model(checkpoint, mode, split, augment, out_test_dir, npy_file)
        npy_file_to_sbmit_csv(mode, split, npy_file, csv_file)



# main #################################################################
if __name__ == '__main__':

    run_test_fold()


    print('\nsucess!')





--- [START 2018-11-11_18-30-12] ----------------------------------------------------------------

	SEED         = 35202
	PROJECT_PATH = C:\Users\Admin\Desktop\doodle\code
	out_test_dir = ../split/test

** dataset setting **

test_dataset : 
	split        = <NIL>
	mode         = test
	complexity   = simplified
	len(self.id) = 112199



** net setting **
Load imagenet pretrained SERESNEXT50
<class 'model_seresnext50.Net'>


	checkpoint = ../../output/backup/00126000_model_96color_r2.pth
	 0 256 512 768 1024 1280 1536 1792 2048 2304 2560 2816 3072 3328 3584 3840 4096 4352 4608 4864 5120 5376 5632 5888 6144 6400 6656 6912 7168 7424 7680 7936 8192 8448 8704 8960 9216 9472 9728 9984 10240 10496 10752 11008 11264 11520 11776 12032 12288 12544 12800 13056 13312 13568 13824 14080 14336 14592 14848 15104 15360 15616 15872 16128 16384 16640 16896 17152 17408 17664 17920 18176 18432 18688 18944 19200 19456 19712 19968 20224 20480 20736 20992 21248 21504 21760 22016 22272 22528 22784 23040 23296 2

In [3]:
import pandas as pd

In [4]:
file = pd.read_csv('../split/test/test-null.submit.csv.gz')

FileNotFoundError: [Errno 2] No such file or directory: '../split/test/test-null.submit.csv.gz'